In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import open3d as o3d
import k3d

from utils import *
from ray_tracing import *

## Test ray_tracing.intersection_points

In [2]:
# init the object and normalize
bun = o3d.data.BunnyMesh()
mesh_bun = o3d.io.read_triangle_mesh(bun.path)
mesh_bun.vertices = o3d.utility.Vector3dVector(normalize(np.asarray(mesh_bun.vertices)))

# init camera views
camera_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=2, resolution=10)
views = np.asarray(camera_sphere.vertices)

In [3]:
# setup open3d virtual camera
width = 320
height = 240
vis, ctr, param = setup_virtual_camera(width, height)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [4]:
# calculcate camera fov in degrees
fx = param.intrinsic.intrinsic_matrix[0, 0]
x0 = param.intrinsic.intrinsic_matrix[0, 2]
fov = np.arctan2(x0, fx) * 180 / np.pi

In [5]:
# setup raycasting scene and calculate the rays
scene = o3d.t.geometry.RaycastingScene()
center = np.zeros(3)
up = np.array([0, 1, 0])
eye = views[7]
rays = scene.create_rays_pinhole(fov, center, eye, up, width, height)
rays = rays.numpy().reshape(-1, 6)
rays = Ray(origin=Vec3D(rays[:, :3]), direction=Vec3D(rays[:, 3:]))

In [6]:
# setup the grid
num_voxels = 32
voxel_size = 2. / num_voxels
min_bound = Vec3D(np.array([-1., -1., -1.]))
max_bound = Vec3D(-min_bound.data)
grid = Grid(voxel_size, num_voxels, min_bound, max_bound)

In [7]:
# find and process t_min and t_max
t_min, t_max = intersection_points(rays, grid)
# mask out the non-intersecting rays
min_mask = np.logical_or(t_min == np.inf, t_min == -np.inf)
max_mask = np.logical_or(t_max == np.inf, t_max == -np.inf)
mask = np.logical_not(np.logical_or(min_mask, max_mask))
print(t_min.min(), t_min.max())
# clip rays within a threshold
thresh = 4  # twice the radius
t_min = np.clip(t_min, a_min=0, a_max=thresh)
t_max = np.clip(t_max, a_min=0, a_max=thresh)

In [8]:
# plot the object
vertices = flip_axes(np.asarray(mesh_bun.vertices))
indices = np.asarray(mesh_bun.triangles)
plot = k3d.plot()
plot += k3d.mesh(
            vertices, 
            indices,
            attribute=vertices[:, 2],
            color_map=k3d.colormaps.matplotlib_color_maps.inferno,
        )

# plot entrance points
points = flip_axes(rays.origin.data + t_min[:, None] * rays.direction.data)[mask]
plot += k3d.points(points, point_size=0.01, shader='mesh', color=0xff0ac5)

# plot exit points
points = flip_axes(rays.origin.data + t_max[:, None] * rays.direction.data)[mask]
plot += k3d.points(points, point_size=0.01, shader='mesh', color=0x000ac5)

# plot camera
plot += k3d.points(flip_axes(eye), point_size=0.1, shader='mesh', color=0x00ffc5)

plot.display()

/Users/khasmamad/miniconda3/envs/nbv/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
/Users/khasmamad/miniconda3/envs/nbv/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "int32" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Output()